In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import KFold,train_test_split,GridSearchCV,StratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

In [37]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
sample=pd.read_csv('sample_submission.csv')

In [38]:
dat=pd.concat([train,test])

In [39]:
train.head()

,Area_Code,Locality_Code,Region_Code,Height,Diameter,Class,Species
0,4694,17,1609,3.0,5.0,7,48
1,781,7,1380,4.0,17.0,2,54
2,21014,6,1122,3.0,11.0,1,22
3,7326,6,556,3.0,34.0,7,25
4,13122,17,1752,5.0,16.0,2,128


In [40]:
train['Class']=train['Class'].apply(str)

In [83]:
X.shape[1]

26

In [44]:
dum=pd.get_dummies(dat['Locality_Code'],prefix='Locality')
dat=pd.concat([dat,dum],axis=1)


In [45]:
dat

,Area_Code,Locality_Code,Region_Code,Height,Diameter,Class,Species,Locality_0,Locality_1,Locality_2,...,Locality_12,Locality_13,Locality_14,Locality_15,Locality_16,Locality_17,Locality_18,Locality_19,Locality_20,Locality_21
0,4694,17,1609,3.0,5.0,7.0,48,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,781,7,1380,4.0,17.0,2.0,54,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21014,6,1122,3.0,11.0,1.0,22,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7326,6,556,3.0,34.0,7.0,25,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13122,17,1752,5.0,16.0,2.0,128,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29550,20833,1,1170,5.5,11.0,NaN,77,0,1,0,...,0,0,0,0,0,0,0,0,0,0
29551,28896,12,834,3.0,9.0,NaN,156,0,0,0,...,1,0,0,0,0,0,0,0,0,0
29552,2193,20,2169,7.5,32.0,NaN,49,0,0,0,...,0,0,0,0,0,0,0,0,1,0
29553,25195,10,1425,8.0,35.0,NaN,28,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
px.bar(y=train.groupby(['Class'])['Area_Code'].count(),color=train['Class'].unique())

In [73]:
px.bar(y=train.groupby(['Class'])['Species'].sum(),color=train['Class'].unique())

In [66]:
px.bar(y=train.groupby(['Class'])['Height'].mean(),color=train['Class'].unique())

In [67]:
px.bar(y=train.groupby(['Class'])['Diameter'].mean(),color=train['Class'].unique())

In [63]:
px.scatter(y=train.groupby(['Class'])['Height'].mean(),x=train.groupby(['Class'])['Diameter'].mean(),color=train['Class'].unique())

In [68]:
px.box(train,x='Class',y='Height',color='Class')

In [51]:
px.box(train,x='Class',y='Diameter',color='Class')

In [70]:
px.scatter(train,x='Diameter',y='Height',facet_col='Class',width=1000,color='Class')

In [46]:
train1=dat.iloc[:len(train)]
test1=dat.iloc[len(train):]

In [47]:
test1.drop(['Class'],axis=1,inplace=True)

In [91]:
X=train1.drop(['Area_Code','Region_Code','Class'],axis=1)
y=train1['Class']

In [88]:
from skopt.space import Real,Integer,Categorical
from sklearn.metrics import make_scorer,log_loss,accuracy_score
from sklearn.model_selection import cross_val_score

In [92]:
from skopt.utils import use_named_args
reg=RandomForestClassifier(n_estimators=256,)
space  = [Integer(1, 16, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(1,X.shape[1] , name='max_features'),
          Integer(,name='n_estimators')]
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=5, n_jobs=-1,
                                    scoring=make_scorer(accuracy_score)))

In [93]:
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=50, random_state=0)

"Best score=%.4f" % res_gp.fun

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [49]:
from sklearn.ensemble import RandomForestClassifier,StackingClassifier

In [70]:
model=StackingClassifier(estimators=[('rf',RandomForestClassifier())],cv=5,final_estimator=LGBMClassifier())
model.fit(X,y)

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       max_samples=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                        

In [31]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 12, 'n_estimators': 256, 'num_leaves': 16}

In [71]:
preds=model.predict_proba(test1.drop(['Area_Code','Region_Code'],axis=1))

In [72]:
preds

array([[3.68795406e-04, 4.50480092e-03, 3.28813388e-01, ...,
        3.13824767e-05, 6.39741743e-01, 3.97174187e-04],
       [1.93534389e-03, 1.86386544e-02, 2.00499924e-02, ...,
        1.71701065e-04, 1.59624244e-03, 9.55093544e-01],
       [4.67649289e-02, 3.75468896e-04, 8.10044918e-03, ...,
        8.30700163e-03, 1.04892797e-01, 3.31359334e-04],
       ...,
       [2.39059606e-03, 2.94662738e-03, 1.46106898e-01, ...,
        2.00460672e-04, 5.94189587e-01, 5.24450371e-04],
       [1.66975052e-03, 6.44182091e-04, 1.77338762e-02, ...,
        1.15912787e-04, 4.36856961e-01, 4.54224223e-04],
       [7.04229343e-03, 2.06721909e-04, 1.32980716e-03, ...,
        6.42267575e-03, 1.70803994e-02, 1.82436508e-04]])

In [73]:
cols=sample.columns
sample[cols]=preds

In [74]:
sample

,Class_0,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7
0,0.000369,0.004505,0.328813,0.026123,0.000020,0.000031,0.639742,0.000397
1,0.001935,0.018639,0.020050,0.002503,0.000011,0.000172,0.001596,0.955094
2,0.046765,0.000375,0.008100,0.831213,0.000015,0.008307,0.104893,0.000331
3,0.014612,0.089358,0.009906,0.002628,0.000013,0.000359,0.001592,0.881532
4,0.001631,0.059485,0.641456,0.011917,0.000028,0.000042,0.278534,0.006908
...,...,...,...,...,...,...,...,...
29550,0.011194,0.147688,0.494395,0.042443,0.000035,0.000042,0.256226,0.047976
29551,0.001606,0.438953,0.063442,0.033722,0.000143,0.000031,0.006666,0.455437
29552,0.002391,0.002947,0.146107,0.253618,0.000024,0.000200,0.594190,0.000524
29553,0.001670,0.000644,0.017734,0.542505,0.000020,0.000116,0.436857,0.000454


In [75]:
sample.to_csv('pred6.csv',index=False)